<a href="https://colab.research.google.com/github/EddyDavies/Trade_With_Twitter/blob/main/ipynb/Transformer_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check GPU

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Check TPU

Check Memory

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [3]:
!ls 

Transformer_Inference.ipynb     text_extraction_with_bert.ipynb


In [4]:
 # !cp -r  ../../drive/MyDrive/ColabData/bitcoin_tweets ../data

In [5]:
%cd Trade_With_Twitter/src
!ls

In [6]:
from google.colab import drive

drive.mount('/content/drive')

In [7]:
!pip install -r ../requirements.txt


In [16]:
import os
import pandas as pd
from tqdm import tqdm

from utils import get_date_array, get_month_array
from sentiment.inference import get_tweets, get_sentiments, to_dict_of_lists, save_sentiments, get_paths

source, results_folder = get_paths()
# source, results_folder = get_paths(data_folder='../../drive/MyDrive/ColabData')

In [10]:

dates_range = ("2017-01-01", "2021-06-20")
months_range = ["Jan 17", "Jun 21"]

months = get_month_array(months_range)
dates = get_date_array(dates_range)

In [11]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis")

In [13]:
from sentiment.inference import check_last_day

for date in dates:
    ids, tweets = get_tweets(date, source)

    results = get_sentiments(date, ids, tweets, sentiment_analysis, 
                             save_every=10, reduction=10)



    outputs = to_dict_of_lists(results)
    outputs["ids"] = ids
    save_sentiments(outputs, date)

    check_last_day(results, results_folder, date)


FileNotFoundError: [Errno 2] No such file or directory: '../data/bitcoin_scores/Jan 17/MTurk_2017-01-01.csv'

In [15]:
!ls ../

data             ipynb            src
examples         requirements.txt venv


In [ ]:
!rm -r ../data/bitcoin_scores

In [17]:
print(results_folder)

../data/bitcoin_scores/
